In [ ]:
!pip install gwcloud-python bilby

In [ ]:
from gwcloud_python import GWCloud

### user token -- sign into gwcloud.org.au/auth/api-token.  Token is unique to individuals and provides authentication e.g., to LVC data and cluster resources
gwc = GWCloud(token='PlaceYourTokenHere') 

In [ ]:
from gwcloud_python import Cluster

#We now submit a new job to GWCloud using the standard ini file. The ini file will specify everything GWCloud needs to know
#to run the parameter estimation.
spin_job = gwc.start_bilby_job_from_file(
    job_name = "GW150914 With Spin",
    job_description = "Testing GWCloud",
    private=True,
    ini_file='GW150914.ini',
    cluster = Cluster.CIT
)

no_spin_job = gwc.start_bilby_job_from_file(
    job_name = "GW150914 With Spin",
    job_description = "Testing GWCloud",
    private=True,
    ini_file='NoSpin.ini',
    cluster = Cluster.CIT
)

In [ ]:
#Check on the status of the jobs
print(spin_job.status)
print(no_spin_job.status)

In [ ]:
import gwcloud_python
#To obtain the log bayes factor and create the superimposed pltos we must download the merge json file (size roughly 1G) and 
#read the result from this file using bilby.core.result.read_in_result(filename)

jobs = gwc.get_user_jobs[-2:]
for job in jobs:
    json_files = job.get_full_file_list().filter_list_by_path(directory='result', extension='json')
    file_list = gwcloud_python.file_reference.FileReferenceList()
    file_list.append(json_files[0])
    job.save_files_by_reference(file_list, './result')

In [ ]:
import bilby

#Here we create the result objects from the result files. Now, we can make plots and read data.

spin_result = bilby.core.result.read_in_result(filename=str(spin_json_files[0].path))
no_spin_result = bilby.core.result.read_in_result(filename=str(no_spin_json_files[0].path))

print(spin_result.log_bayes_factor)
print(no_spin_result.log_bayes_factor)

bilby.core.result.plot_multiple([spin_result, no_spin_result], labels = ['Spin', 'No Spin'], parameters = ['mass_1', 'mass_2', 'mass_ratio', 'theta_jn', 'luminosity_distance'])
